---

### **脑MRI的图像格式**

---

#### **引言：MRI图像的采集与影响因素**  
MRI（磁共振成像）是一项复杂的医学成像技术，其图像质量受**硬件性能**（如磁场均匀性、线圈灵敏度）、**扫描参数**（体素大小、TR重复时间、TE回波时间、翻转角等）和**受试者状态**（运动伪影、生理噪声）共同影响。例如：  
- **体素大小**：决定图像分辨率（如1mm³体素比3mm³更清晰）。  
- **TR/TE**：影响图像对比度（T1加权、T2加权等）。  
- **扫描序列**：如梯度回波（GRE）与平面回波（EPI）适用于不同功能成像。  

然而，本教程**不深入探讨成像原理**，而是聚焦于**图像格式的解析与转换**。若对MRI原理感兴趣，推荐学习权威网站：[MRI Questions](https://mriquestions.com/index.html)。  

---

### **第一部分：DICOM——原始图像的存储格式**

#### **1.1 DICOM格式的来源与特点**  
- **来源**：MRI扫描仪直接输出的原始数据，每个扫描序列生成数百至数千个`.dcm`文件。  
- **特点**：  
  - **多文件结构**：每个切片（slice）保存为独立文件，包含**成像参数**（如层厚、磁场强度）和**患者信息**。  
  - **标准化协议**：DICOM（Digital Imaging and Communications in Medicine）是医学影像的国际通用格式。  
  - **数据冗余**：原始数据体积庞大（单次扫描可达数GB），需转换为其他格式便于分析。  
  - **信息泄露**：容易出现用户个人数据的信息泄露
![DICOM图像](png/DICOM.png)
 

#### **1.2 DICOM的局限性**  
- **处理复杂**：多文件管理不便，需专用软件（如MRIcroGL、dcm2niix、MRIcro或者RadiAnt DICOM Viewer）读取。  
- **兼容性差**：直接用于科研分析效率低，需转换为**NIFTI**格式。  

---

### **第二部分：NIFTI——标准化体素空间**

#### **2.1 NIFTI格式的优势**  
- **三维/四维数据**：将多个DICOM文件整合为单个`.nii`或`.nii.gz`（压缩）文件，支持时间序列（如fMRI）。  
- **标准化坐标**：包含**体素-物理空间映射**（如原点位置、方向矩阵），便于跨设备数据对齐。  
- **广泛兼容**：主流工具（FSL、SPM、AFNI）均支持NIFTI。  
![NII 图像](png/NII.png)

#### **2.2 NIfTI图像的基本结构与空间坐标系**

NIfTI格式的核心在于其**头文件（Header）**中定义的坐标系信息，这些信息决定了如何将离散的体素（voxel）索引转换为真实的物理空间坐标（以毫米为单位）。理解这些内容对后续的图像配准、分析和可视化至关重要。



##### **(1). 体素空间与物理空间的关系**
每个NIfTI图像本质上是一个**三维数组**，其中每个元素（体素）的索引为 `(i, j, k)`，对应其在数组中的行列层位置。然而，这些索引需要通过**仿射变换**（Affine Transformation）映射到物理空间坐标 `(x, y, z)`。  
这一映射关系由头文件中的两个关键字段定义：  
- **`qform`（四元数变换）**：使用四元数（quaternion）描述旋转和平移。  
- **`sform`（仿射矩阵变换）**：直接通过仿射矩阵（4×4）定义体素到物理空间的映射。  

若两者同时存在，**`sform` 的优先级高于 `qform`**。



##### **(2). 坐标系定义**
NIfTI的物理空间坐标系遵循以下规则（以标准模板为例）：  
- **原点 `(0, 0, 0)`**：通常定义为**前连合（Anterior Commissure, AC）**与中线（midline）的交点（即AC点）。  
- **坐标轴方向**：  
  - **Y轴**：沿大脑中线，从后（负方向）向前（正方向）延伸，穿过AC点。  
  - **Z轴**：垂直于Y轴，从下（负方向）向上（正方向）延伸，穿过大脑纵裂。  
  - **X轴**：垂直于Y和Z轴，从左（负方向）向右（正方向）延伸，穿过AC点。  

这一坐标系设计使得**左侧大脑对应X负方向**，**右侧对应X正方向**，符合医学影像的通用标准。


##### **(3). `qform` 与 `sform` 的数学表达**
NIfTI通过以下矩阵实现体素索引到物理坐标的转换：  
```math
\begin{bmatrix} x \\ y \\ z \\ 1 \end{bmatrix} = 
\begin{bmatrix}
R_{11} & R_{12} & R_{13} & T_x \\
R_{21} & R_{22} & R_{23} & T_y \\
R_{31} & R_{32} & R_{33} & T_z \\
0 & 0 & 0 & 1
\end{bmatrix}
\cdot
\begin{bmatrix} i \\ j \\ k \\ 1 \end{bmatrix}
```
其中：
- **`R`** 为旋转矩阵，由 `qform` 的四元数或 `sform` 直接定义。
- **`T`** 为平移向量，表示原点偏移。

###### **`qform` 的特点**  
- 使用四元数（`quatern_b, quatern_c, quatern_d`）和偏移参数（`qoffset_x, qoffset_y, qoffset_z`）定义旋转和平移。  
- 适用于简单的刚体变换（无缩放或错切）。  

###### **`sform` 的特点**  
- 直接存储完整的仿射矩阵（`srow_x, srow_y, srow_z`），支持任意线性变换（如缩放、旋转、平移、错切）。  
- 更灵活，常用于多模态数据配准。  


##### **(4). 如何读取空间信息？**
通过Python库（如`nibabel`）可快速获取NIfTI的坐标系参数：
```python
import nibabel as nib

img = nib.load('example.nii')
header = img.header

# 获取仿射矩阵（sform优先）
affine = img.affine  # 4x4矩阵

# 单独读取qform和sform
qform = img.get_qform()
sform = img.get_sform()
```


##### **(5). 注意事项**
- **原点可能不唯一**：实际数据中，原点位置取决于扫描仪设置或配准模板，需通过仿射矩阵确认。  
- **单位一致性**：NIfTI默认单位为毫米（mm），但需检查头文件的`xyzt_units`字段确认。  

通过理解NIfTI的坐标系与变换矩阵，可确保图像在空间分析（如配准、统计）中的准确性！这个部分仅作为对图像基本特征和数据结构形式的理解，更具体地操作内容将逐步展开！
#### **2.3 体素空间的局限性**  
- **形态学不匹配**：大脑皮层是高度折叠的曲面，而NIFTI的**立方体素**（3D网格）无法贴合其解剖结构，导致配准（Registration）和分析误差。  
- **冗余信息**：包含非目标区域（如头骨、脑脊液），需额外步骤（如掩膜提取）剔除。  

**总结**：NIFTI适用于全脑体素分析（如VBM），但皮层研究需更贴合解剖的**表面数据格式**。  

---

### **第三部分：皮层空间——Freesurfer与GIFTI**

---

#### **3.1 Freesurfer格式详解**

Freesurfer通过分割MRI数据生成**表面模型**（灰质与白质边界、脑脊液边界等），其核心是使用**三角网格**（Triangle Mesh）描述皮层曲面。具体的分割原理我们这里不详述，其核心过程是将白质和灰质以及其他脑组织进行精确分割，分割后，使用白质结构作为基础，通过拓扑学算法重建皮层表面，例如下图：

![ freesurfer重建 ](png/surface_recon.jpg)


其曲面数据实际为一个一个小三角构成，将其称为vertex，中文翻译一般为顶点：


![ 皮层表面 ](png/surface_mesh.png)


**以下为关键文件类型及其内容**：

##### **3.1.1 几何文件：`white` 与 `pial`**
- **文件内容**：  
  - **顶点坐标**：每个顶点（vertex）是一个三维点，表示皮层表面的位置（单位为毫米）。  
  - **面片索引**：每个面片（face）由3个顶点索引组成，定义三角网格的连接关系。  
- **数据存储**：  
  - `white`：白质表面（灰质与白质交界）。  
  - `pial`：软脑膜表面（灰质与脑脊液交界）。  
- **示例结构**：  
  ```python
  # 使用nibabel读取Freesurfer表面文件
  from nibabel.freesurfer import read_geometry

  vertices, faces = read_geometry('lh.white')  # 左半球白质表面
  print(f"顶点数: {vertices.shape}")  # (N, 3)，N为顶点数量（如163842）
  print(f"面片数: {faces.shape}")     # (M, 3)，M为三角面片数量（如327680）
  ```


##### **3.1.2 功能数据文件：`.mgh` 与 `.mgz`**
- **文件内容**：  
  - **标量值数组**：每个顶点对应一个数值（如皮层厚度、激活强度）。  
  - **元数据**：包含空间坐标系信息（如体积-表面配准矩阵）。  
- **数据存储**：  
  - `.mgh`：未压缩的二进制格式。  
  - `.mgz`：gzip压缩后的格式（体积更小）。  
- **示例读取**：  
  ```python
  from nibabel import load

  mgh_data = load('lh.thickness.mgz')  
  data_array = mgh_data.get_fdata()  # 形状为 (1, 1, 1, N)，N为顶点数
  ```


##### **3.1.3 标注文件：`.annot` 与 `.label`**
- **`.annot` 文件**：  
  - **脑区划分**：每个顶点分配一个标签（如默认模式网络、视觉网络）。  
  - **颜色表**：标签与颜色的映射关系（如Desikan-Killiany脑图谱）。  
- **`.label` 文件**：  
  - **兴趣区标记**：仅包含特定区域（如海马体）的顶点索引列表。  
- **示例读取**：  
  ```python
  from nibabel.freesurfer import read_annot, read_label

  # 读取.annot文件
  labels, ctab, names = read_annot('lh.aparc.annot')  

  # 读取.label文件
  label_indices = read_label('lh.hippocampus.label')  
  ```


##### **3.1.4 表面空间与模板：fsaverage 与 sphere**  
- **fsaverage模板**：  
  - **标准表面**：Freesurfer预定义的配准模板，支持不同分辨率：  
    - `fsaverage3`：约3k顶点/半球（低分辨率）。  
    - `fsaverage4`：约6k顶点/半球。  
    - `fsaverage5`：约10k顶点/半球。  
    - `fsaverage6`：约41k顶点/半球。  
    - `fsaverage`：约163k顶点/半球（高分辨率）。  
  - **用途**：跨被试数据对齐到同一空间。  
- **sphere文件**：  
  - **球面配准**：将个体皮层表面映射到球面，便于跨被试配准。  
  - 文件示例：`lh.sphere.reg`（左半球配准球面）。  


##### **3.1.5 网格数据本质：顶点与面片**  
- **顶点（Vertex）**：三维坐标点，定义曲面位置。  
- **面片（Face）**：由3个顶点组成的三角形，构建连续曲面。  
- **优势**：  
  - 更贴合皮层折叠形态（相比立方体素）。  
  - 支持表面配准、曲率分析等高级计算。  


#### **3.2 GIFTI格式详解**

##### **3.2.1 GIFTI与Freesurfer的对应关系**  
GIFTI作为标准化格式，可直接存储Freesurfer生成的数据：  
| **Freesurfer格式** | **GIFTI子类型**       | **内容**                  |  
|---------------------|-----------------------|---------------------------|  
| `lh.white`          | `surf.gii`            | 几何表面（顶点+面片）      |  
| `lh.thickness.mgz`  | `func.gii`            | 顶点标量值（连续数据）     |  
| `lh.aparc.annot`    | `label.gii`           | 顶点标签（离散分类）       |  

##### **3.2.2 GIFTI的优势**  
1. **标准化**：跨平台兼容（Freesurfer、Connectome Workbench、Python）。  
2. **元数据支持**：可嵌入数据来源、单位、坐标系描述。  
3. **高效压缩**：支持GZip压缩（`.gii.gz`）。  


##### **3.2.3 Python读取GIFTI文件**  
使用`nibabel`库可直接解析GIFTI数据：  
```python
import nibabel as nib

# 读取几何表面
surf_gii = nib.load('lh.white.surf.gii')
vertices = surf_gii.agg_data('pointset')  # 顶点坐标 (N, 3)
faces = surf_gii.agg_data('triangle')     # 面片索引 (M, 3)

# 读取功能数据
func_gii = nib.load('lh.thickness.func.gii')
thickness = func_gii.darrays[0].data      # 标量值数组 (N,)

# 读取标签数据
label_gii = nib.load('lh.aparc.label.gii')
labels = label_gii.darrays[0].data       # 标签数组 (N,)
```

##### **3.2.4 格式转换示例**  
将Freesurfer表面数据转换为GIFTI：  
```bash
# 使用Freesurfer命令mris_convert
mris_convert lh.white lh.white.surf.gii
```

#### **总结**  
- **Freesurfer**：提供完整的皮层建模工具链，但格式封闭。  
- **GIFTI**：作为开放标准，支持跨工具数据交换，尤其适合表面分析。  
- **Python生态**：通过`nibabel`和`nilearn`可无缝处理两类格式。  

**优势**：GIFTI直接存储顶点数据（非体素），更贴合皮层形态，适用于表面分析（如脑网络拓扑）。  

---



### **第四部分：CIFTI——融合体素与皮层的HCP格式**

#### **4.1 CIFTI的核心设计**

CIFTI（Connectivity Informatics Technology Initiative）格式由人类连接组计划（HCP）提出，旨在统一**皮层表面数据**与**亚皮层体素数据**，解决传统体素或表面格式的局限性。



##### **4.1.1 数据整合与Grayordinates**
- **混合坐标系**：  
  CIFTI文件通过`Grayordinates`（灰质坐标）将两类数据整合：  
  - **皮层表面顶点**：每个半球约32,000个顶点（基于`fsLR`模板的32k分辨率）。  
  - **亚皮层体素**：约10,000个体素，覆盖基底节、丘脑等灰质区域。  
- **文件结构**：  
  - 数据存储为二维矩阵，维度为`(Grayordinates数量 × 时间点/特征)`。  
  - 扩展名为`.nii`，但内部结构包含CIFTI特有元数据，需专用工具解析。  



##### **4.1.2 文件类型与用途**  
| **文件类型**       | **用途**                              | **示例场景**                     |  
|---------------------|---------------------------------------|----------------------------------|  
| `dtseries.nii`      | 时间序列数据（如fMRI、EEG）            | 静息态功能连接分析                |  
| `dscalar.nii`       | 标量数据（如皮层厚度、激活强度）        | 组间差异映射                     |  
| `dlabel.nii`        | 标签数据（如脑网络分区）                | 默认模式网络划分                 |  
| `ptseries.nii`      | 经过投影的时间序列（如从体素到表面）    | 跨模态数据融合                   |  



#### **4.2 CIFTI的优势**

##### **4.2.1 高效存储**  
- **数据精简**：仅保留灰质相关区域（剔除白质、脑脊液），减少存储冗余。  
- **压缩支持**：支持GZip压缩（如`dtseries.nii.gz`），进一步缩小文件体积。  

##### **4.2.2 跨模态融合**  
- **表面-体素混合分析**：  
  - 可同时计算**皮层顶点间功能连接**与**皮层-亚皮层连接**。  
  - 示例：研究视觉皮层（表面）与丘脑（体素）的协同活动。  



#### **4.3 CIFTI的依赖模板：fsLR与fsaverage**

##### **4.3.1 fsLR模板**  
- **双重坐标参考**：  
  - **体素空间**：对应MNI152NLin6Asym标准空间。  
  - **表面空间**：基于`fsaverage_LR`模板（左右半球对称配准）。  
- **多分辨率支持**：  
  | **分辨率** | 顶点数/半球 | 应用场景                      |  
  |------------|-------------|------------------------------|  
  | 164k       | 163,842     | HCP 3T/7T结构像分析          |  
  | 59k        | 59,412      | HCP 7T功能像/DWI分析         |  
  | 32k        | 32,492      | HCP 3T功能像/DWI分析         |  
  | 4k         | 4,096       | MEG/EEG源重建分析            |  

##### **4.3.2 模板转换与降采样**  
- **跨分辨率分析**：  
  - 使用`Connectome Workbench`工具实现数据降采样（如32k→4k）。  
  - 示例命令：  
    ```bash
    wb_command -cifti-resample input.dscalar.nii 32k 4k BARYCENTRIC output.dscalar.nii
    ```  
- **空间依赖关系**：  
  CIFTI的皮层坐标需基于`fsaverage`或`fsLR`模板生成，具体配准方法将在后续**空间标准化**部分详解。


#### **4.4 如何读取CIFTI文件？**

##### **Python示例**  
使用`nibabel`库解析CIFTI数据：  
```python
import nibabel as nib

# 读取CIFTI文件
cifti = nib.load('sub-01_task-rest.dtseries.nii')
data = cifti.get_fdata()  # 形状为 (Grayordinates数量, 时间点)

# 提取元数据
brain_models = cifti.header.get_axis(1)  # Grayordinates结构信息
for model in brain_models:
    if model.name == 'CIFTI_STRUCTURE_CORTEX_LEFT':
        left_cortex_indices = model.vertex  # 左半球顶点索引
```


#### **4.5 注意事项**  
- **工具依赖**：需安装`Connectome Workbench`或`nibabel>=3.0`以支持CIFTI读写。  
- **模板一致性**：跨研究比较时需统一模板分辨率（如均使用32k）。  



### **总结**  
CIFTI通过整合表面与体素数据，为多模态脑影像分析提供了统一框架。其核心依赖`fsLR`模板的高效空间表达，使研究者能够更灵活地探索全脑灰质的功能与结构特性。关于模板配准与空间转换的细节，将在后续章节深入讨论。  

**扩展阅读**：  
- [BIDS坐标系规范](https://bids-specification.readthedocs.io/en/v1.2.2/99-appendices/08-coordinate-systems.html#template-based-coordinate-systems)  
- [HCP CIFTI文档](https://www.humanconnectome.org/software/connectome-workbench) 


### **第五部分：格式转换工具**
本部分仅介绍工具信息，具体的使用必须以实际场景为基础进行练习才能有更深理解和体会，因此具体操作于教程后续内容的不同步骤完成，主要是空间标准化步骤和可视化步骤！
#### **5.1 常用工具与库**  
- **DICOM转NIFTI**：  
  - `dcm2niix`：命令行工具，自动解析DICOM元数据。  
  - `pydicom`（Python库）：自定义读取DICOM文件。  
- **NIFTI转GIFTI/CIFTI**：  
  - `Connectome Workbench`：HCP官方工具，支持复杂转换。  
  - `neuromaps`（Python库）：提供标准化模板对齐与数据重采样。  

#### **5.2 转换注意事项**  
- **坐标系匹配**：确保目标模板（如MNI152）与数据空间一致。  
- **数据插值**：表面与体素数据转换时需选择合适算法（如最近邻/线性插值）。  

---

### **总结：如何选择格式？**  
- **全脑体素分析** → NIFTI  
- **皮层表面研究** → Freesurfer/GIFTI  
- **多模态融合分析** → CIFTI  

通过理解格式特性，可显著提升脑影像分析的效率与准确性！